In [ ]:
import requests
from pyquery import PyQuery as pq
import pytz
from tqdm import tqdm

import windprobabilitynames   # Local file

import datetime
from glob import glob
from importlib import reload
import logging
import re
import os
import sys

In [ ]:
namedict = windprobabilitynames.namedict
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)
reload(logging)

In [ ]:
localtimezonename = "US/Eastern"              # Configure your time zone here
localtimezonenameforpeople = "Eastern"

winddirmain = "/var/www/html/windprobabilities/"
if not os.path.exists(winddirmain):
    winddirmain = "windprobabilities/"

for localdir in [winddirmain]:
    os.makedirs(localdir, exist_ok=True)
    
localtz = pytz.timezone(localtimezonename)

In [ ]:
def sconvert(s):
    if s == "34":
        return("39")
    elif s == "50":
        return("57")
    elif s == "64":
        return("74")
    else:
        print(f"Something went weird with speed conversion on string {s}")

In [ ]:
def winddescribe():
    output = "The National Hurricane Center is estimating the likelihood of powerful windspeeds at a number of locations. "
    output += "Tropical storm speeds begin around 39 mph. 57 mph is an average tropical storm. 74 mph is a Category 1 hurricane."
    return(output)

In [ ]:
def windprobparser(html):
    separator = "\r\n"
    tab = "    "
    divider = "LOCATION       KT"
    ender = "$$"
    mysplitter = "----" + separator
    
    if "<pre>" in html:
        holder = pq(html)("pre").html().split("\n")
    else:
        holder = html.split("\n")
    sourcedate = None
    
    for row in holder:
        if " UTC " in row:
            try:
                row = row.strip()       # Lose whitespace
                utc = pytz.utc
                # eastern = pytz.timezone('US/Eastern')
                sourcedate = utc.localize(datetime.datetime.strptime(row, "%H%M UTC %a %b %d %Y"))
                # simpletimestamp = sourcedate.strftime("%I %p").replace("AM", "a.m.").replace("PM", "p.m.")
                # if simpletimestamp[0] == "0":
                #     simpletimestamp = simpletimestamp[1:]              # Strip off leading zeroes, as from 0500
                # simpletimestamp += f" Eastern windspeed forecast{separator}"
            except:
                sourcedate = None

    if not sourcedate:   # If we were able to unable to translate UTC
        for row in holder:
            if " Z TIME" in row:
                sourcedate = row.strip()

    for i, row in enumerate(holder):
        if divider in row:
            break
    rows = holder[i + 1:]    # Trim off header junk
    for i, row in enumerate(rows):
        if ender in row:      # Trim off stuff after the end of the records
            break
    rows = rows[:i]

    running = ""
    lastname = ""
    output = []
    unknowncities = []
    for row in rows:
        if len(row.strip()) > 10:
            name = row[:14].strip()
            speed = row[14:17].strip()
            percentage = row[row.rfind("(")+1:].replace(")", "").strip()
            if name in namedict:
                name = namedict[name]
            else:
                # print(name)
                unknowncities.append(name)
            if name != "badstuff":
                if name != lastname:   # If we need to start a new row
                    output.append(running)
                    lastname = name
                    running = f"{name}: {percentage}% chance of {sconvert(speed)}+ mph"
                else:
                    running += f"; {percentage}% chance of {sconvert(speed)}+ mph"   # append to existing row
    output.append(running)    # Add the last line
    # print(unknowncities)
    output = list(filter(None, output))     # Remote empty strings
    logger.debug(f"{len(output):,} entries found; {len(unknowncities):,} unknown cities found; timestamp of {sourcedate}")
    return(output, unknowncities, sourcedate)

In [ ]:
def get_latest_winds():
    windpage = "https://ftp.nhc.noaa.gov/atcf/wndprb/?C=N;O=A"
    windbaseurl = "https://ftp.nhc.noaa.gov/atcf/wndprb/"
    utc = pytz.utc
    activecutoff = utc.localize(datetime.datetime.utcnow()-datetime.timedelta(hours=8))    
    r = requests.get(windpage)
    section = pq(r.content)("pre").html().splitlines()
    utc = pytz.utc
    stormwindindex = {}
    for row in section:
        if "wndprb" in row:
            filename = pq(row)("a").attr("href")
            stormname = filename.split(".")[0]
            timestampraw = row.split("</a>")[-1].strip()
            timestampraw = "T".join(timestampraw.split(" ")[:2])
            timestamp = datetime.datetime.strptime(timestampraw, "%Y-%m-%dT%H:%M")
            timestamp = utc.localize(timestamp)
            stormwindurl = windbaseurl + filename
            if timestamp > activecutoff:
                active = True
            else:
                active = False
            stormwindindex[stormname] = {
                "filename": filename,
                "timestamp": timestamp,
                "timestampraw": timestampraw,
                "stormwindurl": stormwindurl,
                "active": active,
            }
    return stormwindindex


In [ ]:
def formatAPDate(date_object, wantyear=False, wantweekday=False):
    import datetime
    if date_object.month == 9:
        new_date = "Sept. " + datetime.datetime.strftime(date_object, "%d").lstrip("0")
    elif date_object.month < 3 or date_object.month > 7:
        new_date = datetime.datetime.strftime(date_object, "%b. ") + datetime.datetime.strftime(date_object, "%d").lstrip("0")
    else:
        new_date = datetime.datetime.strftime(date_object, "%B ") + datetime.datetime.strftime(date_object, "%d").lstrip("0")
    if wantweekday:
        new_date = datetime.datetime.strftime(date_object, "%A, ") + new_date
    if wantyear:
        new_date = new_date + datetime.datetime.strftime(date_object, ", %Y")
    return new_date

In [ ]:
def formatAPTime(date_object, minuteswanted=True):
    import datetime
    new_time = str(int(date_object.strftime("%I")))    # Strip off any leading zeroes
    rawminutes = date_object.strftime("%M")
    if rawminutes != "00" and minuteswanted:
        new_time = f"{new_time}:{date_object.strftime('%M')}"
    flag = date_object.strftime("%p").replace("AM", " a.m.").replace("PM", " p.m.")
    new_time += flag
    return(new_time)

In [ ]:
def formatAPInteger(number_object, minusword="negative ", zeroword="no"):
    replacements = {0: zeroword, 1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six", 7: "seven", 8: "eight", 9: "nine"}
    mytext = ""
    if number_object < 0:
        mytext += minusword
    myabs = abs(number_object)
    if myabs in replacements:
        mytext += replacements[myabs]
    else:
        mytext += f"{myabs:,}"    # Add commas
    return mytext

In [ ]:
def numberthenword(number_object, word, minusword="negative ", zeroword="no", initialcapital=False):
    import inflect
    p = inflect.engine()
    mytext = ""
    mytext += formatAPInteger(number_object, minusword="negative ", zeroword="no")
    mytext += " "      # Space between number and word
    mytext += p.plural(word, number_object)
    if initialcapital:
        mytext = mytext[0].upper() + mytext[1:]
    return mytext

In [ ]:
def build_namekey():
    # Adapted from https://github.com/Unidata/HurricaneTracker/blob/master/hur_tracker.py
    # Key from https://ftp.nhc.noaa.gov/atcf/docs/tcdevel.dat

    # This will almost certainly break next time there's a South Atlantic storm. The basins are named differently.

    abbreviationlookup = {
        "DB": "Disturbance",
        "EX": "Extratropical Cyclone",
        "HU": "Hurricane", 
        "LO": "Low",
        "TC": "Tropical Cyclone",
        "TD": "Tropical Depression",
        "TS": "Tropical Storm",
        "TY": "Typhoon",
        "SD": "Subtropical Depression",
        "SS": "Subtropical Storm",
        "ST": "Super Typhoon",
        "WV": "Tropical Wave",
        "XX": ""
        }

    namekeyurl = "https://ftp.nhc.noaa.gov/atcf/index/storm_list.txt"
    r = requests.get(namekeyurl)
    namekey = {}
    for row in r.text.splitlines():
        fields = row.split(",")
        if len(fields) > 3:
            line = {}
            name = fields[0].strip().title()
            basin = fields[1].strip()
            cyclonenumber = fields[7].strip()
            stormyear = fields[8].strip()
            stormtype = fields[9].strip()
            stormid = basin.lower() + cyclonenumber + stormyear
            if stormtype in abbreviationlookup:
                stormtypefull = abbreviationlookup[stormtype]
            else:
                stormtypefull = ""
            namefull = f"{stormtypefull} {name}".strip()
            if name == "Invest":
                namefull = f"{name} {cyclonenumber}{basin[-1].upper()}"
            for item in ["stormid", "name", "namefull", "basin", "cyclonenumber", "stormyear", "stormtype", "stormtypefull"]:
                line[item] = eval(item)
            namekey[stormid] = line
    return namekey

In [ ]:
# def generate_active_winds():
localtimestamp = datetime.datetime.now().astimezone(localtz)
stormwindindex = get_latest_winds()
activestorms = []
for stormid in stormwindindex:
    if stormwindindex[stormid]["active"] == True:
        activestorms.append(stormid)
logger.debug(f"{len(activestorms):,} active storms found: {' '.join(activestorms)}")
namekey = build_namekey()
for stormid in activestorms:
    targetfilename = f"{winddirmain}{stormid}.html"
    if stormid in namekey:
        name = namekey[stormid]["name"]
        fullname = namekey[stormid]["namefull"]
    else:
        name, namefull = stormid, stormid
        logger.error(f"A storm with the ID of {stormid} is not found in a lookup table.")
    filetimestamp = stormwindindex[stormid]["timestamp"].astimezone(localtz)
    stormwindurl = stormwindindex[stormid]["stormwindurl"]
    r = requests.get(stormwindurl)
    output, unknowncities, sourcedate = windprobparser(r.text)

    if len(output) == 0:    # If we have anything to say, we'll overwrite any previous files
        logger.warning(f"No usable locations found for {stormid} at {stormwindurl}. Not generating a file.")
    else:
        logger.debug(f"{len(output):,} usable locations found for {stormid}")
        html = f"<html><head><title>{fullname} wind probabilities</title></head><body>"
        html += '<p><a href="https://www.hurricanes.gov" target="_blank" rel="noopenner noreferrer">The National Hurricane Center</a> listed the chances of '
        html += f"{fullname}'s effects on a number of locations. Tropical storm speeds begin around 39 mph. "
        html += "57 mph is an average tropical storm. 74 mph is a Category 1 hurricane. "
        html += "The odds are for sustaine winds of a certain speed; gusts can be higher.</p>"
        html += "<ul>"
        for entry in output:
            html += f"<li>{entry}</li>"
        html += "</ul>"
        html += '<p><font size="-1">'
        html += f"File checked at {formatAPTime(localtimestamp)} {localtimezonenameforpeople} from "
        html += '<a target="_blank" rel="noopenner noreferrer" href="' + stormwindurl + '">the source file</a> dated '
        html += f"{formatAPTime(filetimestamp)} {formatAPDate(filetimestamp, wantweekday=True)} {localtimezonenameforpeople}.</font></p>"
        html += "</body></html>"
        with open(targetfilename, "w", encoding="utf-8") as outfile:
            outfile.write(html)

targetfilename = f"{winddirmain}!!index.html"
html = "<html><head><title>"
html += numberthenword(len(activestorms), "active storm", initialcapital=True)
html += " tracked</title></head><body><h1>" + numberthenword(len(activestorms), "active storm", initialcapital=True) 
html += " tracked</h1>"
html += "<p>As of "
html += f"{formatAPTime(localtimestamp)} {localtimezonenameforpeople} {formatAPDate(localtimestamp, wantweekday=True)}:</p><ul>"
for stormid in activestorms:
    filetimestamp = stormwindindex[stormid]["timestamp"].astimezone(localtz)
    fullname = namekey[stormid]["namefull"]
    html += f"<li>{fullname} at "
    html += '<a target="_blank" rel="noopenner noreferrer" href="' + stormid + '.html">' + stormid + "</a>."
    html += f" ... Last file update:  {formatAPTime(filetimestamp)} {localtimezonenameforpeople}</li>"
html += "</ul>"
html += "<p>Note: Storms in the middle of the water may have no file, as they're not affecting anything.</p>"
html += "</body></html>"
with open(targetfilename, "w", encoding="utf-8") as outfile:
    outfile.write(html)
print("Done!")    